# 1. จัดรูปแบบและ clean ข้อมูล ยอดซื้อขาย
ข้อมูลที่นำมาวิเคราะห์เป็นยอดซื้อขายของสมาชิกในร้านค้าสะดวกซื้อ สองสาขา เป็นเวลา 3 เดือน ตั้งแต่เดือน สิงหาคม ถึง ตุลาคม 
ข้อมูลที่ได้เป็นข้อมูล Excel จำนวนสองข้อมูลจากต่างสาขา ที่ประกอบไปด้วย ข้อมูลสมาชิก, เลขทีบิล, วันที่, รายละเอียดสินค้าที่ซื้อ, จำนวนสินค้า, ราคาเต็ม และส่วนลด

In [1]:
pip install xlrd

In [2]:
!pip install -U -q PyDrive

In [3]:
# โค้ดใช้ plot graph
import seaborn as sns
import warnings
# โค้ดจัดการตาราง
import pandas as pd
import numpy as np
# set maximum row to unlimit
#pd.options.display.max_rows = 999
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", None, "display.max_columns", None) #show entire dataframe

## 1.1 นำเข้าข้อมูลไฟล์จาก google drive 

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [5]:
# นำเข้าข้อมูลสาขาที่ A 
file_id = '1CCR7h-2NSZ5HKhkGIJ-5qE5-tH0NpDdB'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data1.xlsx')
dat1 = pd.read_excel('data1.xlsx', skiprows= 4)
dat1.head()

,วันที่,เลขที่บิล,จำนวน,ขายก่อนลด,ส่วนลด,ลดสมาชิก,ขายรวมภพ.,ขายสุทธิ.,Unnamed: 8,Unnamed: 9
0,สมาชิก,01000029,นางXXXX,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-08-01 00:00:00,900701056314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,200.93,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# นำเข้าข้อมูลสาขาที่ B
file_id = '1pUMj8kJ9rn-93uparz42JmxTDVDgwfUm'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data2.xlsx')
dat2 = pd.read_excel('data2.xlsx', skiprows= 3)
dat2.head()

,วันที่,เลขที่บิล,จำนวน,ขายก่อนลด,ส่วนลด,ลดสมาชิก,ขายรวมภพ.,ขายสุทธิ.
0,สมาชิก,01000043,บ. XXXX จำกัด,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-04 00:00:00,700201020159,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1.2 จัดระเบียบข้อมูลในตาราง

### ลบบรรทัดและคอลัมน์ที่ว่างในตาราง และเพิ่มคอลัมน์สาขา (branch)

In [7]:
#Drop the rows where all elements are missing.
# ข้อมูลสาขา A
#ลบบรรทัดว่าง (NaN) ที่อยู่คอลัมน์เลขที่บิล ในตาราง
dat1.dropna(subset=['เลขที่บิล'], inplace= True)
dat1.drop(columns=['ขายสุทธิ.','Unnamed: 8','Unnamed: 9'], inplace= True)
dat1['Branch'] = 'A'
print(dat1.shape)
dat1.head()

(1981, 8)


,วันที่,เลขที่บิล,จำนวน,ขายก่อนลด,ส่วนลด,ลดสมาชิก,ขายรวมภพ.,Branch
0,สมาชิก,01000029,นางXXXX,NaN,NaN,NaN,NaN,A
1,2020-08-01 00:00:00,900701056314,NaN,NaN,NaN,NaN,NaN,A
3,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A
5,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A
7,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A


In [8]:
#ลบบรรทัดว่าง (NaN) ที่อยู่คอลัมน์เลขที่บิล ในตาราง
# ข้อมูลสาขา B
dat2.dropna(subset=['เลขที่บิล'], inplace= True)
dat2.drop(columns=['ขายสุทธิ.'], inplace= True)
dat2['Branch'] = 'B'
print(dat2.shape)
dat2.head()

(1657, 8)


,วันที่,เลขที่บิล,จำนวน,ขายก่อนลด,ส่วนลด,ลดสมาชิก,ขายรวมภพ.,Branch
0,สมาชิก,01000043,บ. XXXX จำกัด,NaN,NaN,NaN,NaN,B
3,2020-08-04 00:00:00,700201020159,NaN,NaN,NaN,NaN,NaN,B
5,223027161008,AA อร่อยดี กะทิอบควันเทียน250มล.,1,755.0,0.0,0.0,755.0,B
7,223522021072,AA มิตรผล น้ำตาลทรายแดง1กก.,14,7560.0,0.0,0.0,7560.0,B
9,038197600032,AB ข้าวเหนียวดำ500ก.ไร่ทิพย์,1,280.0,0.0,0.0,280.0,B


In [9]:
# merge A,B branch dataframe into one dataframe
dat = pd.concat([dat1,dat2],ignore_index=True)
print(dat.shape)
dat.head()

(3638, 8)


,วันที่,เลขที่บิล,จำนวน,ขายก่อนลด,ส่วนลด,ลดสมาชิก,ขายรวมภพ.,Branch
0,สมาชิก,01000029,นางXXXX,NaN,NaN,NaN,NaN,A
1,2020-08-01 00:00:00,900701056314,NaN,NaN,NaN,NaN,NaN,A
2,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A
3,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A
4,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A


###เพิ่มคอลัมน์ เลขสมาชิก และ ระบุเพศของสมาชิกจากคำนำหน้า


In [10]:
#กำหนดชื่อคอลัมน์ใหม่
dat.columns = ['Product_id','Product_detail','Amount','Price','Discount','Discount_member','Sale_price','Branch']
dat['Date'],dat['Invoice_id'],dat['Member_id'],dat['Gender'] = [np.nan, np.nan, np.nan, np.nan] 
dat.head()

,Product_id,Product_detail,Amount,Price,Discount,Discount_member,Sale_price,Branch,Date,Invoice_id,Member_id,Gender
0,สมาชิก,01000029,นางXXXX,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN
1,2020-08-01 00:00:00,900701056314,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN
2,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A,NaN,NaN,NaN,NaN
3,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A,NaN,NaN,NaN,NaN
4,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A,NaN,NaN,NaN,NaN


### สร้าง dataframe 'member' แล้วไปลงรายละเอียดคอลัมน์ Member_id และ Gender
เลขสมาชิกอยู่ในบรรทัดที่มีคำว่า 'สมาชิก' ในคอลัมน์ Product_id 


In [11]:
member = dat.loc[dat.Product_id == 'สมาชิก']
member.Member_id = member.Product_detail
member.drop(member.columns[[0,1,3,4,5,6,7,8,9]],axis=1,inplace=True) # ลบคอลัมน์ที่ไม่จำเป็นออกไป
member.columns = ['Name', 'Member_id', 'Gender' ]
member

,Name,Member_id,Gender
0,นางXXXX,01000029,NaN
477,นางสาวXXXX,02000173,NaN
659,น.ส.XXXXX,02000404,NaN
1155,นายXXXXX,02000986,NaN
1465,นายXXXXXX,02000993,NaN
1574,นายXXXXX,02001135,NaN
1647,นายXXXXXX,02213348,NaN
1739,น.ส. XXXXX,03004515,NaN
1780,นายXXXXX,038841706,NaN
1977,น.ส.XXXXXX,038846327,NaN


### ระบุเพศสมาชิกจากคำนำหน้าชื่อ

In [12]:
member.Gender[member.Name.str.contains('นาง')| member.Name.str.contains('น.ส.')] = 'Female'
member.Gender[member.Name.str.contains('นาย')] = 'Male'
member

,Name,Member_id,Gender
0,นางXXXX,01000029,Female
477,นางสาวXXXX,02000173,Female
659,น.ส.XXXXX,02000404,Female
1155,นายXXXXX,02000986,Male
1465,นายXXXXXX,02000993,Male
1574,นายXXXXX,02001135,Male
1647,นายXXXXXX,02213348,Male
1739,น.ส. XXXXX,03004515,Female
1780,นายXXXXX,038841706,Male
1977,น.ส.XXXXXX,038846327,Female


In [13]:
# ระบุ บ.XXX เป็นบริษัท และคุณXXXX เป็นผู้หญิง
member.Gender[member.Member_id == '01000043'] = 'Company'
member.Gender[member.Member_id == '03011705'] = 'Female'
member

,Name,Member_id,Gender
0,นางXXXX,01000029,Female
477,นางสาวXXXX,02000173,Female
659,น.ส.XXXXX,02000404,Female
1155,นายXXXXX,02000986,Male
1465,นายXXXXXX,02000993,Male
1574,นายXXXXX,02001135,Male
1647,นายXXXXXX,02213348,Male
1739,น.ส. XXXXX,03004515,Female
1780,นายXXXXX,038841706,Male
1977,น.ส.XXXXXX,038846327,Female


In [14]:
#ระบุรายละเอียดสมาชิกในข้อมูล dat
for ind in member.index.tolist():
  dat.Member_id.loc[ind:] = member.Member_id.loc[ind]
  dat.Gender.loc[ind:] = member.Gender.loc[ind]
dat.head()

,Product_id,Product_detail,Amount,Price,Discount,Discount_member,Sale_price,Branch,Date,Invoice_id,Member_id,Gender
0,สมาชิก,01000029,นางXXXX,NaN,NaN,NaN,NaN,A,NaN,NaN,01000029,Female
1,2020-08-01 00:00:00,900701056314,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,01000029,Female
2,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A,NaN,NaN,01000029,Female
3,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A,NaN,NaN,01000029,Female
4,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A,NaN,NaN,01000029,Female


In [15]:
# ลบบรรทัดรายละเอียดสมาชิกออกไปเพราะเราระบุในคอลัมน์ member_id แล้ว
dat = dat.loc[ ((dat['Product_id'] != 'สมาชิก') & (dat.Product_id != 'รวมสมาชิก')) ]
dat.head()

,Product_id,Product_detail,Amount,Price,Discount,Discount_member,Sale_price,Branch,Date,Invoice_id,Member_id,Gender
1,2020-08-01 00:00:00,900701056314,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,01000029,Female
2,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A,NaN,NaN,01000029,Female
3,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A,NaN,NaN,01000029,Female
4,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A,NaN,NaN,01000029,Female
5,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A,NaN,NaN,01000029,Female


### ระบุวันและเลข invoice ของแต่ละสินค้า

In [16]:
import datetime
invoice_date = dat[dat['Product_id'].apply(lambda x: isinstance(x, datetime.datetime))][['Product_id','Product_detail','Member_id']]
invoice_date.columns = ['Date','Invoice','Member_id']
invoice_date.head()

,Date,Invoice,Member_id
1,2020-08-01 00:00:00,900701056314,01000029
20,2020-08-02 00:00:00,600401172634,01000029
31,2020-08-02 00:00:00,900701056577,01000029
42,2020-08-04 00:00:00,901101026599,01000029
60,2020-08-05 00:00:00,901101026613,01000029


In [17]:
#ระบุรายละเอียดวันที่และ invoice ใน dat1_df
for ind in invoice_date.index.tolist():
  dat.Date.loc[ind:] = invoice_date.Date.loc[ind]
  dat.Invoice_id.loc[ind:] = invoice_date.Invoice.loc[ind]
dat.head()

,Product_id,Product_detail,Amount,Price,Discount,Discount_member,Sale_price,Branch,Date,Invoice_id,Member_id,Gender
1,2020-08-01 00:00:00,900701056314,NaN,NaN,NaN,NaN,NaN,A,2020-08-01 00:00:00,900701056314,01000029,Female
2,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A,2020-08-01 00:00:00,900701056314,01000029,Female
3,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A,2020-08-01 00:00:00,900701056314,01000029,Female
4,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A,2020-08-01 00:00:00,900701056314,01000029,Female
5,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A,2020-08-01 00:00:00,900701056314,01000029,Female


In [18]:
# ลบบรรทัด invoice ออกไปเพราะเราระบุในคอลัมน์ Invoice_id แล้ว
dat = dat[ ~ dat['Product_id'].apply(lambda x: isinstance(x, datetime.datetime))]
dat.head()

,Product_id,Product_detail,Amount,Price,Discount,Discount_member,Sale_price,Branch,Date,Invoice_id,Member_id,Gender
2,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A,2020-08-01 00:00:00,900701056314,01000029,Female
3,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A,2020-08-01 00:00:00,900701056314,01000029,Female
4,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A,2020-08-01 00:00:00,900701056314,01000029,Female
5,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A,2020-08-01 00:00:00,900701056314,01000029,Female
6,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A,2020-08-01 00:00:00,900701056314,01000029,Female


## 1.3 จัดประเภทสินค้าเป็นหมวดหมู่ 
ได้แก่ Snack, Grains & noodles, Canned & Jarred goods, Beverages & Drink,Dry/Baking Goods , Spices and Sauces ,Personal Care , Others

### เนื่องจากข้อมูลสินค้าไม่ได้แยกประเภทมาให้ จึงต้องนำข้อมูลชื่อสินค้ามาวิเคราะห์ว่ามีศัพท์คำไหนที่พบบ่อย เพื่อทำฟังก์ชันในการแยกประเภทสินค้าจากคำศัพท์ต่างๆ



In [19]:
product_word = np.concatenate(dat.Product_detail.str.split().values)
len(product_word)

10946

In [20]:
product_word[1:50]

array(['เพียวริคุ', 'ชาขาวมิกซ์เบอร์รี่', '350มล.', 'AA', 'โออิชิ',
       'น้ำผึ้งผสมมะนาว180มล.', 'AA', 'เพียวริคุ', 'ชาขาวมิกซ์เบอร์รี่',
       '350มล.', 'AA', 'โออิชิ', 'น้ำผึ้งผสมมะนาว180มล.', 'AA', 'โออิชิ',
       'น้ำผึ้งผสมมะนาว180มล.', 'AA', 'เนสกาแฟ', 'เอ็กซ์ตร้า180', 'มล.',
       'AA', 'เพียวริคุ', 'ชาขาวคูลผสมเก็กฮวย', '350มล.', 'AA',
       'เพียวริคุ', 'ชาขาวคูลผสมเก็กฮวย', '350มล.', 'AA', 'เพียวริคุ',
       'ชาขาวคูลผสมเก็กฮวย', '350มล.', 'AA', 'เพียวริคุ',
       'ชาขาวมิกซ์เบอร์รี่', '350มล.', 'AA', 'องุ่น',
       'น้ำมันถั่วเหลืองขวด', '1ลิตร', 'AA', 'องุ่น',
       'น้ำมันถั่วเหลืองขวด', '1ลิตร', 'AA', 'ไชโย', 'น้ำมันปาล์ม',
       'ถุง1ลิตร', 'AA'], dtype='<U47')

In [21]:
# นับจำนวนคำที่เจอใน รายละเอียดชื่อสินค้า
pd.value_counts(product_word).head(50)

AA               2055
AAB               249
1กก.              138
A                 120
เลย์              114
มล.                95
50กรัม             95
กรัม               87
แป้งทอดกรอบ        73
55กรัม             68
ชูรสอายิโนะ        65
มาม่า              64
13กรัม             64
700มล.             56
AB                 55
เลย์ร็อค           53
600มล.             53
(เรียบ)            50
น้ำมันปาล์ม        49
180มล.             49
150มล.             47
350มล.             46
300มล.             45
250มล.             45
100                42
ซอสหอยนางรม        42
450มล.             41
กะทิ               41
อร่อยดี            40
โอวัลติน           38
น้ำตาลทรายขาว      38
รสโนริสาหร่าย      37
ยูเอฟเอ็ม          37
1000มล.            36
รสดี               36
500กรัม            36
75ก.               36
รสดีรสหมู          36
รสหมู              36
น้ำปลาขวดPET       35
มิตรผล             35
ครัววังทิพย์       34
17กรัม             34
ภูเขาทอง           33
ขนมอบกรอบ          33
80ก.      

## สร้างฟังก์ชันเพื่อแยกประเภทสินค้า
ใช้ Keywords ในการแยกประเภทสินค้า

In [22]:
# keyword list 
# เลือกคำที่พบใน ชื่อสินค้า เพื่อนำมาจัดประเภทสินค้า

Grain_noodles = ['มาม่า', 'ไวไว', 'ยำยำ', 'หมี่', 'โจ๊ก', 'งาดำ',
                 'ถั่ว', 'อัลมอนด์', 'ข้าวเหนียว', 'งาขาว', 'เม็ดแมงลัก',
                 'สาคู', 'ข้าวขาว', 'ข้าวหอม', 'เส้นใหญ่', 'เมล็ด']

Beverages_Drink = ['แลคตาซอย', 'ไวตา', 'ดีมอล', 'ดีโด้', 'น้ำดื่ม',
                   'เบอร์ดี้', 'หนองโพ', 'กาแฟ', 'เครื่องดื่ม', 'เพียวริคุ',
                   'โสมพลัส', 'นมสด', 'ยูนิฟ', 'ดีน่า', 'ชาลิปตัน', 'ดัชมิลล์',
                   'ไทยเดนมาร์ค', 'โฟร์โมส', 'โอวัลติน', 'UHT', 'โออิชิ',
                   'อิชิตัน', 'กระทิงแดง', 'คาราบาว', 'สปอนเซอ', 'เฮลซ์บลูบอย',
                   'ลิโพ', 'เป๊ปซี่', 'แฟนต้า', 'สิงห์', 'เบียร์', 'สปาย', 'โค๊ก',
                   'โค้ก', 'แมนซั่ม', 'แบรนด์', 'ทิบโก้', 'กาโตะ', 'ชาแดง',
                   'ชาเขียว', 'ทิปโก้', 'น้ำหวาน', 'มอคโคน่า', 'คอฟฟี่', 'เครื่องดื่ม',
                   'น้ำบ๊วย', 'C-Vitt', 'ไมโล', 'ไอวี่']

Spices_Sauces = ['ซีอิ๊ว', 'ซีอิ้ว', ' ซอส', 'ซอส ', 'ซอสมะเขือเทศโรซ่',
                    'ซอสหอย', 'เต้าเจี้ยว', 'น้ำส้มสายชู', 'อสร', 'น้ำจิ้ม',
                    'ซอสเปรี้ยว', 'น้ำปลา', 'น้ำพริก', 'น้ำตาลทราย',
                    'น้ำตาลปี๊บ', 'รสดี', 'เกลือปรุง', 'ฟ้าไทย', 'น้ำมัน',
                    'ชูรส', 'น้ำเชื่อม', 'มงกุฎเพชร', 'กะปิ', 'พริกไทย',
                    'น้ำมะขามเปียก', 'ผงหมูแดง', 'ตังไฉ่', 'ผงกะหรี่', 'ซอสเห็ดหอม', 'ซอสพริก']

snack = ['เลย์', 'ปูไทย', 'ขนม', 'ทาโร', 'ขาไก่',
             'เวเฟอร์', 'พริงเกิลล์', 'ฮานามิ', 'มันฝรั่ง',
             'เฟรนด์ฟรายด์', 'กูลิโกะ', 'คุ๊กกี้', 'คุกกี้',
             'โดโซะ', 'ปาปริก้า', 'ทิวลี่', 'ไดโนพาร์ค',
             'ปลาหมึก', 'ชีส', 'เบง', 'บิชชิน', 'ปาร์ตี้',
             'ตะวัน', 'นิชชิน', 'ฟันโอ', 'แคมปัส', 'โอรีโอ',
             'โฮมมี่', 'ทวิสโก้', 'บาร์บีคิว', 'ข้าวเกรียบ',
             'กัสเซ็น', 'โปเต้', 'ปีโป้', 'โยโย', 'ซีมอล',
             'ครีโก้', 'โคอะล่ามาร์ช', 'คอปปบอลส', 'เค้ก',
             'ดิวเบอร์รี่', 'ลอตเต้', 'แมลโลว์', 'ซีมอน', 'วอยซ์', 'ครีมโอ']
Canned_Jarred_goods = ['กะทิ', 'แมคเคอเรล', 'ปลาซาร์ดีน',
                          'สามแม่ครัว', 'ปลากระป๋อง', 'นกพิราบ', 'นมข้น']

Dry_Baking_Goods = ['แป้งข้าว', 'แป้งทอด', 'แป้งมัน',
                       'แป้งสาลี', 'วุ้นเส้น', 'ครีมเทียม',
                       'แป้งเอนกประสงค์', 'แป้งท้าว']

Personal_Care = ['สบู่', 'แชมพู', 'ปรับผ้านุ่ม', 'การ์นิเย่',
                     'น้ำยาล้าง', 'แคร์', 'แป้งเด็ก', 'แป้งเย็น',
                     'โพรเทค', 'ยาสี', 'คอลเกต', 'แปรงสีฟัน',
                     'ลอรีเอะ', 'ไฮเตอร์', 'บีไนซ์', 'ครีมนวด',
                     'โรลออน', 'ไฟน์ไล', 'โซฟี', 'ซันไล', 'ยันฮี',
                     'ตรางู', 'เปา', 'สำลี', 'ซัก', 'แอทแทค',
                     'โอโม', 'บรีส', 'โปร', 'ไฮยีน', 'เบบี้เลิฟ',
                     'มามี่โพโค', 'ซีซีครีม', 'ครีมอาบน้ำ', 'โลชั่น']

Services = ['ค่าบริการ']


Category_key = dict.fromkeys(Grain_noodles, "Grain & Noodles")
Category_key.update(dict.fromkeys(Beverages_Drink, "Beverages & Drink"))
Category_key.update(dict.fromkeys(Spices_Sauces, "Spices & Sauces"))
Category_key.update(dict.fromkeys(snack, "Snacks"))
Category_key.update(dict.fromkeys(Canned_Jarred_goods, "Canned & Jarred goods"))
Category_key.update(dict.fromkeys(Dry_Baking_Goods, "Dry & Baking goods"))
Category_key.update(dict.fromkeys(Personal_Care, "Personal Care"))



In [23]:
# function สำหรับสร้างคอลัมน์ category
def Category_match(Category_dict, data_df):

  # สร้าง คอลัมน์  category แล้วให้ defalt เป็น Other
  data_df['category'] = 'Other'

  for key in Category_dict:
    #print(key)

    # check ว่ามี keyword อยู่ในชื่อสินค้าไหม ถ้ามีให้ใส่ ประเภทสินค้าในคอลัมน์ category
    if any(data_df.Product_detail.str.contains(key)):
      #print(key, Category_dict[key])
      data_df.loc[data_df.Product_detail.str.contains(key), 'category'] = Category_dict[key]

  return(data_df)


In [24]:
dat_output = Category_match(Category_key,dat)
dat_output.head()

,Product_id,Product_detail,Amount,Price,Discount,Discount_member,Sale_price,Branch,Date,Invoice_id,Member_id,Gender,category
2,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A,2020-08-01 00:00:00,900701056314,01000029,Female,Beverages & Drink
3,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A,2020-08-01 00:00:00,900701056314,01000029,Female,Beverages & Drink
4,212547041005,AA เพียวริคุ ชาขาวมิกซ์เบอร์รี่ 350มล.,1,215.0,0.0,0.0,215.0,A,2020-08-01 00:00:00,900701056314,01000029,Female,Beverages & Drink
5,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A,2020-08-01 00:00:00,900701056314,01000029,Female,Beverages & Drink
6,205022031002,AA โออิชิ น้ำผึ้งผสมมะนาว180มล.,1,235.0,0.0,0.0,235.0,A,2020-08-01 00:00:00,900701056314,01000029,Female,Beverages & Drink


## 1.4 ลบรายการสินค้าที่ซื้อแล้วคืนสินค้า
สินค้าที่มีราคาติดลบคือสินค้าที่คืน จึงต้องลบรายการที่ซื้อและคืนออกไป

In [25]:
remove_item = dat_output[dat_output.Amount < 0]
remove_item.head()

,Product_id,Product_detail,Amount,Price,Discount,Discount_member,Sale_price,Branch,Date,Invoice_id,Member_id,Gender,category
182,040102400002,ตั้งฉ่าย(ถุงใหญ่)500ก.ตรากระต่าย,-2,-1080.0,0.0,0.0,-1080.0,A,2020-08-19 00:00:00,901101026785,01000029,Female,Other
208,207956011002,AA แบรนด์ ซุปไก่เขียวเล็ก1.5ออนซ์,-1,-2145.0,0.0,0.0,-2145.0,A,2020-08-23 00:00:00,900701060202,01000029,Female,Beverages & Drink
311,133394011013,ปะทัดตาแป๊ะ,-1,-90.0,0.0,0.0,-90.0,A,2020-08-30 00:00:00,900701061386,01000029,Female,Other
313,250981031003,อิชิตัน รสข้าวญี่ปุ่น 420มล.,-4,-1220.0,0.0,0.0,-1220.0,A,2020-08-30 00:00:00,900701061386,01000029,Female,Beverages & Drink
405,216012011076,AA มาม่า ต้มยำน้ำข้น แพ็ค6,-1,-885.0,0.0,0.0,-885.0,A,2020-09-17 00:00:00,900701065110,01000029,Female,Grain & Noodles


In [26]:
len(dat_output)

3293

In [27]:
# หา index ของ item ที่ซื้อแล้วคืน แล้วไปลบสินค้านั้นออก
 

for i in remove_item.index.tolist():
    
    Product_id = remove_item.Product_id.loc[i]
    Amount = (-1) * remove_item.Amount.loc[i]
    member_id = remove_item.Member_id.loc[i]
    #print(Product_id,Amount,remove_item.Product_detail.loc[i])
    item_toremove = dat_output[(dat_output.Product_id == Product_id) & (dat_output.Amount == Amount)
                              & (dat_output.Member_id == member_id)].index.tolist()
    if len(item_toremove) != 0:
        #print(item_toremove[0])
        dat_output.drop(item_toremove, inplace = True)
    #remove one order of each item
    dat_output.drop(i, inplace = True)
        
len(dat_output)

3210

In [28]:
# เช็คว่ายังมีสินค้าติดลบอยู่ไหม
dat_output[dat_output.Amount < 0]


,Product_id,Product_detail,Amount,Price,Discount,Discount_member,Sale_price,Branch,Date,Invoice_id,Member_id,Gender,category


## 1.5 บันทึกข้อมูลเป็นตารางไฟล์ CSV เพื่อทำไปทำ Data visualization ใน section ถัดไป

In [ ]:
from google.colab import  drive
drive.mount('/drive') # Mounts the google drive to Colab Notebook
dat_output.to_csv('/drive/My Drive/DSE_G1/Week1_Data_analytic/Supermarket_sale_data.csv', index = False)

